In [ ]:
# 기본 모듈 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
# path 지정
base_path = "/content/drive/MyDrive/Colab Notebooks/data/235687_시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/convert/"
save_path = "/content/drive/MyDrive/Colab Notebooks/data/235687_시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/result/"

In [ ]:
# train data 불러오기
train_quality = pd.read_csv(base_path+'train_quality_data_.csv')
train_problem = pd.read_csv(base_path+'train_problem_data_.csv')
train_error = pd.read_csv(base_path+'train_err_data_.csv')
test_quality = pd.read_csv(base_path+'test_quality_data_.csv')
test_error = pd.read_csv(base_path+'test_err_data_.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (5,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# 모델링 (GradientBoosting)

### train_df 생성

In [ ]:
# train_error
id_error = train_error[['user_id', 'errtype']].values
error = np.zeros((15000,42))

for person_idx, err in tqdm(id_error):
    # person_idx -10000 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - 10000, err - 1] += 1

error

100%|██████████| 16554663/16554663 [00:58<00:00, 280932.88it/s]


array([[  0.,   0.,   8., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 113.,  56.,   1.],
       [  0.,   0.,   2., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  58.,   8.,   5.],
       [  0.,   0.,   0., ...,   6.,   0.,   0.],
       [  0.,   0.,   4., ...,   0.,   0.,   0.]])

In [ ]:
train_df = pd.DataFrame(error)
train_df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,...,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,...,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,...,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,...,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0


### target 생성

In [ ]:
# train_problem
problem = np.zeros(15000)
problem[train_problem.user_id.unique()-10000] = 1

display(problem, sum(problem))

array([0., 1., 0., ..., 1., 1., 0.])

5000.0

In [ ]:
target = pd.DataFrame(problem)
target

,0
0,0.0
1,1.0
2,0.0
3,0.0
4,1.0
...,...
14995,0.0
14996,0.0
14997,1.0
14998,1.0


### Gradient Boosting 훈련

#### try 1

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# train validation 분리
X_train, X_val, y_train, y_val = train_test_split(train_df,target,test_size=0.2, random_state=42)
X_train.shape,X_val.shape,y_train.shape,y_val.shape

((12000, 42), (3000, 42), (12000, 1), (3000, 1))

In [ ]:
y_train

,0
9839,0.0
9680,0.0
7093,0.0
11293,0.0
820,0.0
...,...
5191,1.0
13418,1.0
5390,1.0
860,1.0


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

model = GradientBoostingClassifier(n_estimators = 41,
                                   learning_rate = 0.1483)

model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(learning_rate=0.1483, n_estimators=41)

In [ ]:
# Validation 데이터셋 예상
pred = model.predict(X_val)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

display(confusion_matrix(y_val, pred))  #confusion matrix 표시
print(precision_score(y_val, pred)) #precision(정밀도)
print(recall_score(y_val, pred))     #sensitivity(민감도), recall(재현율)
print(f1_score(y_val, pred))         # F1 score(정밀도, 민감도 조화평균)
print(classification_report(y_val, pred))

array([[1555,   78],
       [ 811,  556]])

0.8769716088328076
0.4067300658376006
0.5557221389305347
              precision    recall  f1-score   support

         0.0       0.66      0.95      0.78      1633
         1.0       0.88      0.41      0.56      1367

    accuracy                           0.70      3000
   macro avg       0.77      0.68      0.67      3000
weighted avg       0.76      0.70      0.68      3000



#### try2

In [ ]:
model2 = GradientBoostingClassifier(n_estimators = 41,
                                   learning_rate = 0.1483)

model2 = model2.fit(X_train,y_train)
# Validation 데이터셋 예상
pred2 = model2.predict(X_val)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

display(confusion_matrix(y_val, pred2))  #confusion matrix 표시
print(precision_score(y_val, pred2)) #precision(정밀도)
print(recall_score(y_val, pred2))     #sensitivity(민감도), recall(재현율)
print(f1_score(y_val, pred2))         # F1 score(정밀도, 민감도 조화평균)
print(classification_report(y_val, pred2))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[1555,   78],
       [ 810,  557]])

0.8771653543307086
0.4074615947329919
0.5564435564435564
              precision    recall  f1-score   support

         0.0       0.66      0.95      0.78      1633
         1.0       0.88      0.41      0.56      1367

    accuracy                           0.70      3000
   macro avg       0.77      0.68      0.67      3000
weighted avg       0.76      0.70      0.68      3000



#### try3

In [ ]:
model3 = GradientBoostingClassifier(n_estimators = 41,
                                   learning_rate = 0.1483,
                                    #loss='log_loss', 
                                    # learning_rate=0.1, 
                                    # n_estimators=100, 
                                    subsample=0.95, 
                                    # criterion='friedman_mse', 
                                    # min_samples_split=2, 
                                    # min_samples_leaf=1, 
                                    # min_weight_fraction_leaf=0.0, 
                                    max_depth=3, 
                                    # min_impurity_decrease=0.0, 
                                    # init=None, 
                                    # random_state=None, 
                                    # max_features=None, 
                                    # verbose=0, 
                                    # max_leaf_nodes=None, 
                                    # warm_start=False, 
                                    # validation_fraction=0.1, 
                                    # n_iter_no_change=None, 
                                    # tol=0.0001, 
                                    # ccp_alpha=0.0
                                    )

model3 = model3.fit(X_train,y_train)
# Validation 데이터셋 예상
pred3 = model3.predict(X_val)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

display(confusion_matrix(y_val, pred2))  #confusion matrix 표시
print(precision_score(y_val, pred2)) #precision(정밀도)
print(recall_score(y_val, pred2))     #sensitivity(민감도), recall(재현율)
print(f1_score(y_val, pred2))         # F1 score(정밀도, 민감도 조화평균)
print(classification_report(y_val, pred2))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[1542,   91],
       [ 763,  604]])

0.8690647482014389
0.44184345281638626
0.5858389912706111
              precision    recall  f1-score   support

         0.0       0.67      0.94      0.78      1633
         1.0       0.87      0.44      0.59      1367

    accuracy                           0.72      3000
   macro avg       0.77      0.69      0.68      3000
weighted avg       0.76      0.72      0.69      3000

